# langchain调用本地开源大模型

1. **类属性定义**:
   - `max_token`: 定义了模型可以处理的最大令牌数。
   - `do_sample`: 指定是否在生成文本时采用采样策略。
   - `temperature`: 控制生成文本的随机性，较高的值会产生更多随机性。
   - `top_p`: 一种替代`temperature`的采样策略，这里设置为0.0，意味着不使用。
   - `tokenizer`: 分词器，用于将文本转换为模型可以理解的令牌。
   - `model`: 存储加载的模型对象。
   - `history`: 存储对话历史。
2. **构造函数**:
   - `__init__`: 构造函数初始化了父类的属性。
3. **属性方法**:
   - `_llm_type`: 返回模型的类型，即`ChatGLM3`。
4. **加载模型的方法**:
   - `load_model`: 此方法用于加载模型和分词器。它首先尝试从指定的路径加载分词器，然后加载模型，并将模型设置为评估模式。这里的模型和分词器是从Hugging Face的`transformers`库中加载的。
5. **调用方法**:
   - `_call`: 一个内部方法，用于调用模型。它被设计为可以被子类覆盖。
   - `invoke`: 这个方法使用模型进行聊天。它接受一个提示和一个历史记录，并返回模型的回复和更新后的历史记录。这里使用了模型的方法`chat`来生成回复，并设置了采样、最大长度和温度等参数。
6. **流式方法**:
   - `stream`: 这个方法允许模型逐步返回回复，而不是一次性返回所有内容。这对于长回复或者需要实时显示回复的场景很有用。它通过模型的方法`stream_chat`实现，并逐块返回回复。

In [1]:
from langchain.llms.base import LLM
from langchain_core.messages.ai import AIMessage
from transformers import AutoTokenizer, AutoModel, AutoConfig


class kk_ChatGLM3(LLM):
    max_token: int = 8192
    do_sample: bool = True
    temperature: float = 0.3
    top_p: float = 0.0
    tokenizer: object = None
    model: object = None
    history: list = []
    
    def __init__(self):
        super().__init__()
        
    @property
    def _llm_type(self):
        return "kk_ChatGLM3"
    
    def load_model(self, model_path: str):
        # 配置分词器
        tokenizer = AutoTokenizer.from_pretrained(model_path, trust_remote_code=True, use_fast=False)
        # 加载模型
        model = AutoModel.from_pretrained(model_path, trust_remote_code=True, device_map="auto")
        model = model.eval()
        self.tokenizer = tokenizer
        self.model = model
        
    def _call(self, prompt: str, config: dict ={}, history: list = []):
        return self.invoke(prompt, config, history)
    
    def invoke(self, prompt: str, config: dict ={}, history: list = []):
        if not isinstance(prompt, str):
            prompt = prompt.is_string()
        
        response, history = self.model.chat(
            self.tokenizer,
            prompt,
            history=history,
            do_sample=self.do_sample,
            max_length=self.max_token,
            temperature=self.temperature,
        )
        self.history = history
        return AIMessage(content=response)
    
    def stream(self, prompt: str, config: dict ={}, history: list = []):
        if not isinstance(prompt, str):
            prompt = prompt.is_string()
        preResponse = ""
        for response, new_history in self.model.stream_chat(self.tokenizer, prompt):
            if preResponse == "":
                result = response
            else:
                result = response[len(preResponse):]
            preResponse = response
            yield result


/home/libing/miniconda3/envs/kk_langchain_wsl/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/libing/miniconda3/envs/kk_langchain_wsl/lib/python3.10/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


In [2]:
llm = kk_ChatGLM3()
model_path = "/home/libing/kk_LLMs/chatglm3-6b-32k"
llm.load_model(model_path)

/home/libing/miniconda3/envs/kk_langchain_wsl/lib/python3.10/site-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/home/libing/miniconda3/envs/kk_langchain_wsl/lib/python3.10/site-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
Loading checkpoint shards: 100%|██████████| 7/7 [00:05<00:00,  1.23it/s]


In [3]:
# 调用call方法

llm.invoke("中国的首都是哪里？")

AIMessage(content='中国的首都是北京。', additional_kwargs={}, response_metadata={})